In [35]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps

import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import requests
import json
import gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [53]:
# source accessed 08/27/20
# https://data.oaklandnet.com/Public-Safety/CrimeWatch-Maps-Past-90-Days/ym6k-rx7a
file = "CrimeWatch_Maps_Past_90-Days.csv"
crime_pd = pd.read_csv(file)
crime_pd.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location
0,EMBEZZLEMENT,07/11/2020 07:24:00 AM,20-033934,THEFT OF UTILITY SERVICES,08X,3400 TELEGRAPH AV,Oakland,CA,"3400 TELEGRAPH AV\nOakland, CA\n(37.822749, -1..."
1,PETTY THEFT,06/02/2020 08:00:00 AM,20-027227,THEFT,33X,900 82ND AV,Oakland,CA,"900 82ND AV\nOakland, CA"
2,BURG - COMMERCIAL,06/02/2020 08:05:00 PM,20-027342,BURGLARY-FORCIBLE ENTRY,08X,3400 TELEGRAPH AV,Oakland,CA,"3400 TELEGRAPH AV\nOakland, CA\n(37.822749, -1..."
3,BURG - AUTO,05/07/2020 02:17:00 PM,20-022949,BURGLARY-AUTO,06X,3000 MARKET ST,Oakland,CA,"3000 MARKET ST\nOakland, CA\n(37.820911, -122...."
4,STOLEN VEHICLE,07/24/2020 03:00:00 PM,20-037615,VEHICLE THEFT - AUTO,31Y,500 85TH AV,Oakland,CA,"500 85TH AV\nOakland, CA\n(37.740242, -122.192..."


In [54]:
# location format
print(crime_pd.iloc[0]['Location'])

3400 TELEGRAPH AV
Oakland, CA
(37.822749, -122.266173)


In [55]:
# isolate coordinates
isolate_coordinate = crime_pd['Location'].str.rsplit('\n(', 1).str.get(1)
isolate_coordinate = isolate_coordinate.str.rsplit(')', 1).str.get(0)
isolate_coordinate

coordinates_pd = pd.DataFrame({"COORDINATES" : isolate_coordinate})

coordinates_split = coordinates_pd["COORDINATES"].str.split(", ", n = 1, expand = True)

crime_pd["LATITUDE"] = coordinates_split[0]
crime_pd["LONGITUDE"] = coordinates_split[1]

crime_pd.head()

,CRIMETYPE,DATETIME,CASENUMBER,DESCRIPTION,POLICEBEAT,ADDRESS,CITY,STATE,Location,LATITUDE,LONGITUDE
0,EMBEZZLEMENT,07/11/2020 07:24:00 AM,20-033934,THEFT OF UTILITY SERVICES,08X,3400 TELEGRAPH AV,Oakland,CA,"3400 TELEGRAPH AV\nOakland, CA\n(37.822749, -1...",37.822749,-122.266173
1,PETTY THEFT,06/02/2020 08:00:00 AM,20-027227,THEFT,33X,900 82ND AV,Oakland,CA,"900 82ND AV\nOakland, CA",NaN,NaN
2,BURG - COMMERCIAL,06/02/2020 08:05:00 PM,20-027342,BURGLARY-FORCIBLE ENTRY,08X,3400 TELEGRAPH AV,Oakland,CA,"3400 TELEGRAPH AV\nOakland, CA\n(37.822749, -1...",37.822749,-122.266173
3,BURG - AUTO,05/07/2020 02:17:00 PM,20-022949,BURGLARY-AUTO,06X,3000 MARKET ST,Oakland,CA,"3000 MARKET ST\nOakland, CA\n(37.820911, -122....",37.820911,-122.276339
4,STOLEN VEHICLE,07/24/2020 03:00:00 PM,20-037615,VEHICLE THEFT - AUTO,31Y,500 85TH AV,Oakland,CA,"500 85TH AV\nOakland, CA\n(37.740242, -122.192...",37.740242,-122.192442


In [56]:
# list different types of crime

crime_type = pd.DataFrame(crime_pd.groupby("CRIMETYPE")['CASENUMBER'].nunique())
crime_type = crime_type.rename(columns = {"CASENUMBER":"COUNT"})
print(crime_type)

                                    COUNT
CRIMETYPE                                
ARSON                                  57
BRANDISHING                            32
BURG - AUTO                          1209
BURG - COMMERCIAL                     412
BURG - OTHER                           53
BURG - RESIDENTIAL                    318
CHILD ABUSE                             9
CURFEW & LOITERING                     50
DISORDERLY CONDUCT                    209
DOMESTIC VIOLENCE                     699
DRUNKENNESS                             4
DUI                                    45
EMBEZZLEMENT                           35
ENVIRONMENTAL CRIME                     1
FELONY ASSAULT                        625
FELONY WARRANT                          6
FORCIBLE RAPE                          50
FORGERY & COUNTERFEITING              299
FRAUD                                 279
GAMBLING                                2
GRAND THEFT                           603
HOMICIDE                          

In [70]:
# isolate HOMICIDE
certain_crimes = ['HOMICIDE']

crime_pd["SELECT_CRIME"] = crime_pd.CRIMETYPE.isin(certain_crimes)

crime_pd_filtered = crime_pd[crime_pd['SELECT_CRIME'] == True]

# remove entries with no coordinates
crime_pd_filtered = crime_pd_filtered.dropna(subset=['LATITUDE', 'LONGITUDE'])

# coordinate pairs only
filtered_coord = pd.DataFrame({"LATITUDE": crime_pd_filtered["LATITUDE"], "LONGITUDE": crime_pd_filtered["LONGITUDE"]})

filtered_coord['LATITUDE'] = filtered_coord['LATITUDE'].astype(float)
filtered_coord['LONGITUDE'] = filtered_coord['LONGITUDE'].astype(float)

filtered_coord.head()

,LATITUDE,LONGITUDE
10,37.766757,-122.168607
77,37.735632,-122.176666
137,37.735632,-122.176666
238,37.793346,-122.211878
294,37.749704,-122.174124


In [75]:
# map out these crimes

# google maps API key redacted

gmaps.configure(api_key="")

fig = gmaps.figure()

symbol = gmaps.symbol_layer(filtered_coord, fill_color='red')
fig.add_layer(symbol)

fig

Figure(layout=FigureLayout(height='420px'))